## Step1 Import Required Packages

In [2]:
# !git clone https://huggingface.co/hfl/rbt3

Cloning into 'rbt3'...
Filtering content:  66% (2/3)
Filtering content: 100% (3/3)
Filtering content: 100% (3/3), 442.86 MiB | 8.88 MiB/s, done.


In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset

d:\anaconda\envs\trans\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Step2 Load the Dataset

In [2]:
dataset = load_dataset("csv", data_files="./data_2.csv", split="train")
dataset = dataset.filter(lambda x: x["content"] is not None and x["label"] is not None and x['label'] in [0,1,2,3,4,5])
#dataset = dataset.filter(lambda x: x["content"] is not None and len(x["content"]) >= 10)
dataset

Dataset({
    features: ['content', 'label'],
    num_rows: 2073
})

## Step3 Split the Dataset

In [3]:
datasets = dataset.train_test_split(test_size=0.1)
datasets

DatasetDict({
    train: Dataset({
        features: ['content', 'label'],
        num_rows: 1865
    })
    test: Dataset({
        features: ['content', 'label'],
        num_rows: 208
    })
})

## Step4 Dataset Preprocessing

In [4]:
import torch

tokenizer = AutoTokenizer.from_pretrained("rbt3")

def process_function(examples):
    tokenized_examples = tokenizer(examples["content"], max_length=128, truncation=True,padding=True)
    #tokenized_examples["labels"] = examples["label"]
    tokenized_examples["labels"] = [int(label) for label in examples["label"]]
    return tokenized_examples

tokenized_datasets = datasets.map(process_function, batched=True, remove_columns=datasets["train"].column_names)
tokenized_datasets

Map: 100%|██████████| 208/208 [00:00<00:00, 10941.98 examples/s]


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 1865
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 208
    })
})

## Step5 Create the model

In [5]:
model = AutoModelForSequenceClassification.from_pretrained("rbt3",num_labels=6)

d:\anaconda\envs\trans\lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at rbt3 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
model.config

BertConfig {
  "_name_or_path": "rbt3",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 3,
  "output_past": true,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version":

## Step6 Create an Evaluation Function

In [7]:
import evaluate

acc_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

In [8]:
def eval_metric(eval_predict):
    predictions, labels = eval_predict
    predictions = predictions.argmax(axis=-1)
    acc = acc_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average='macro')
    acc.update(f1)
    return acc

## Step7 Create TrainingArguments

In [9]:
train_args = TrainingArguments(output_dir="./checkpoints",      # 输出文件夹
                               per_device_train_batch_size=64,  # 训练时的batch_size
                               per_device_eval_batch_size=128,  # 验证时的batch_size
                               logging_steps=10,                # log 打印的频率
                               evaluation_strategy="epoch",     # 评估策略
                               save_strategy="epoch",           # 保存策略
                               save_total_limit=3,              # 最大保存数
                               learning_rate=2e-5,              # 学习率
                               weight_decay=0.1,               # weight_decay
                               metric_for_best_model="f1",      # 设定评估指标
                               load_best_model_at_end=True,
                               num_train_epochs=100)     # 训练完成后加载最优模型
train_args

d:\anaconda\envs\trans\lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_steps=None,
eval_strategy=epoch,
evaluation_strategy=epoch,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsd

## Step8 Create Trainer

In [10]:
from transformers import DataCollatorWithPadding
trainer = Trainer(model=model, 
                  args=train_args, 
                  train_dataset=tokenized_datasets["train"], 
                  eval_dataset=tokenized_datasets["test"], 
                  data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
                  compute_metrics=eval_metric)

## Step9 Model Training

In [11]:
trainer.train()

  0%|          | 12/3000 [00:00<02:56, 16.91it/s]

{'loss': 1.9249, 'grad_norm': 8.071365356445312, 'learning_rate': 1.9933333333333334e-05, 'epoch': 0.33}


  1%|          | 22/3000 [00:01<02:41, 18.48it/s]

{'loss': 1.4553, 'grad_norm': 4.214043140411377, 'learning_rate': 1.9866666666666667e-05, 'epoch': 0.67}


  1%|          | 30/3000 [00:01<02:38, 18.73it/s]

...

                                                   
100%|██████████| 3000/3000 [03:13<00:00, 21.30it/s]

{'eval_loss': 2.00961971282959, 'eval_accuracy': 0.6923076923076923, 'eval_f1': 0.5809239046674044, 'eval_runtime': 0.0853, 'eval_samples_per_second': 2439.033, 'eval_steps_per_second': 23.452, 'epoch': 100.0}


100%|██████████| 3000/3000 [03:13<00:00, 15.50it/s]

{'train_runtime': 193.5287, 'train_samples_per_second': 963.681, 'train_steps_per_second': 15.502, 'train_loss': 0.08951363994088024, 'epoch': 100.0}


TrainOutput(global_step=3000, training_loss=0.08951363994088024, metrics={'train_runtime': 193.5287, 'train_samples_per_second': 963.681, 'train_steps_per_second': 15.502, 'total_flos': 3131102797056000.0, 'train_loss': 0.08951363994088024, 'epoch': 100.0})

## Step10 Model Evaluation

In [12]:
trainer.evaluate(tokenized_datasets["test"])

100%|██████████| 2/2 [00:00<00:00, 65.49it/s]


{'eval_loss': 1.4868144989013672,
 'eval_accuracy': 0.7163461538461539,
 'eval_f1': 0.6139806096773521,
 'eval_runtime': 0.2559,
 'eval_samples_per_second': 812.677,
 'eval_steps_per_second': 7.814,
 'epoch': 100.0}

## Step11 Save Model

In [13]:
# 保存模型、tokenizer、配置文件
trainer.save_model("my_model_label2")
tokenizer.save_pretrained("my_model_label2")

('my_model_label2\\tokenizer_config.json',
 'my_model_label2\\special_tokens_map.json',
 'my_model_label2\\vocab.txt',
 'my_model_label2\\added_tokens.json',
 'my_model_label2\\tokenizer.json')

In [23]:
import pandas as pd
import numpy as np
# 读取 Excel 文件
file_path = 'predict.xls'  # 替换为你的文件路径
df = pd.read_excel(file_path)

# 提取需要预测的列
texts_to_predict = df['内容'].tolist()  # 将内容列转换为列表


In [24]:
model_name = 'my_model_label2'  # 替换为你选择的模型名称，例如 'bert-base-uncased'
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 设置模型为评估模式
model.eval()


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-2): 3 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, e

In [25]:
predictions = []

# 对每个文本进行预测
for text in texts_to_predict:
    # 确保输入是字符串
    if isinstance(text, str):
        # 对文本进行编码
        inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=128)
        
        with torch.no_grad():  # 不需要计算梯度
            outputs = model(**inputs)
        
        # 获取预测的类别 (假设使用 softmax)
        logits = outputs.logits
        predicted_class = torch.argmax(logits, dim=1).item()
        
        predictions.append(predicted_class)
    else:
        predictions.append(0)
        print(f"跳过非字符串输入: {text}")

跳过非字符串输入: nan
跳过非字符串输入: nan
跳过非字符串输入: nan
跳过非字符串输入: nan
跳过非字符串输入: nan
跳过非字符串输入: nan
跳过非字符串输入: nan
跳过非字符串输入: nan
跳过非字符串输入: nan
跳过非字符串输入: nan
跳过非字符串输入: nan
跳过非字符串输入: nan
跳过非字符串输入: nan
跳过非字符串输入: nan
跳过非字符串输入: nan
跳过非字符串输入: nan
跳过非字符串输入: nan
跳过非字符串输入: nan
跳过非字符串输入: nan
跳过非字符串输入: nan
跳过非字符串输入: nan
跳过非字符串输入: nan
跳过非字符串输入: nan
跳过非字符串输入: nan
跳过非字符串输入: nan
跳过非字符串输入: nan
跳过非字符串输入: nan
跳过非字符串输入: nan
跳过非字符串输入: nan
跳过非字符串输入: nan
跳过非字符串输入: nan
跳过非字符串输入: nan
跳过非字符串输入: nan
跳过非字符串输入: nan
跳过非字符串输入: nan
跳过非字符串输入: nan
跳过非字符串输入: nan
跳过非字符串输入: nan
跳过非字符串输入: nan
跳过非字符串输入: nan
跳过非字符串输入: nan
跳过非字符串输入: nan
跳过非字符串输入: nan
跳过非字符串输入: nan
跳过非字符串输入: nan
跳过非字符串输入: nan
跳过非字符串输入: nan
跳过非字符串输入: nan
跳过非字符串输入: nan
跳过非字符串输入: nan
跳过非字符串输入: nan
跳过非字符串输入: nan
跳过非字符串输入: nan
跳过非字符串输入: nan
跳过非字符串输入: nan
跳过非字符串输入: nan
跳过非字符串输入: nan
跳过非字符串输入: nan
跳过非字符串输入: nan
跳过非字符串输入: nan
跳过非字符串输入: nan
跳过非字符串输入: nan
跳过非字符串输入: nan
跳过非字符串输入: nan
跳过非字符串输入: nan
跳过非字符串输入: nan
跳过非字符串输入: nan
跳过非字符串输入: nan
跳过非字符串输入: nan
跳过非字符串输入: nan
跳过非字符串输入: nan
跳过非字符串

In [26]:
# 将预测结果添加到 DataFrame
#df['1=支持中医黑 2=反对中医黑 0=无关'] = predictions
df['主题框架 1=科学论证 2=政治倾向 3=阴谋论 4=文化认同 5=个人经历 0=无关'] = predictions
# 保存到新的 Excel 文件
output_file_path = 'label2.xlsx'  # 输出文件路径
df.to_excel(output_file_path, index=False)